In [1]:
########################################################################################################################
# Overview: This script adjusts the date of record by referencing to the birth date or encounter date of the patient.
########################################################################################################################

In [1]:
########################################################################################################################
# Import packages
########################################################################################################################
import datetime
import gc
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import warnings
from ast import literal_eval
from pyarrow.parquet import ParquetFile
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
########################################################################################################################
# USER_SPECIFIC SETTING
# DATA_IN_DIR_PATH: Path of the input directory of the encounter-level dataset 
# (created in C04_Clean_Patient_Variables.ipynb and C05_Clean_Encounter_Variables.ipynb)
# DICT_IN_FILE_PATH: Path of the input data dictionary file
# (created in C04_Clean_Encounter_Variables.ipynb)
# DICT_OUT_FILE_PATH: Path of the output data dictionary file
########################################################################################################################
DATA_IN_DIR_PATH: str = '../00_Data/01_Cleaned_Data/'
DICT_IN_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v4.xlsx'
DICT_OUT_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v5.xlsx'

In [ ]:
########################################################################################################################
# Load the data dictionary
########################################################################################################################
df_dict_pat: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Patient')
df_dict_enc: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Encounter')

In [ ]:
########################################################################################################################
# [Patient-level, Part 1] Load the data
########################################################################################################################
pat_path: str = os.path.join(DATA_IN_DIR_PATH, 'Patient_full_v1.parquet')
df_pat: pd.DataFrame = pd.read_parquet(pat_path)
print(f'Dimension of the patient-level dataset: {df_pat.shape}')

In [ ]:
########################################################################################################################
# [Patient-level, Part 2] Remove certain unneeded variables (due to small sample size)
########################################################################################################################
remove_keywords: list[str] = ['CommDiseaseExp', 'FreqDrugMisuse', 'SocConnLiving', 'TravelHistory']
pat_vars_to_remove: list[str] = [var for var in df_pat.columns if any(var.startswith(kw) for kw in remove_keywords)]
df_pat.drop(columns=pat_vars_to_remove, inplace=True)
df_dict_pat = df_dict_pat[~df_dict_pat['Variable_Name'].isin(pat_vars_to_remove)]
print(f'{len(pat_vars_to_remove)} variables removed.')

In [ ]:
########################################################################################################################
# [Patient-level, Part 3] For each of the remaining 35 Date variables, make it the days of difference from the BirthDate
########################################################################################################################
date_vars: list[str] = df_dict_pat.loc[df_dict_pat['Variable_Type'] == 'Date', 'Variable_Name'].to_list()
date_vars.remove('BirthDate')
for date_var in date_vars:
    new_var_name: str = f'{date_var}-BirthDate'
    df_dict_pat.loc[df_dict_pat['Variable_Name'] == date_var, 'Variable_Name'] = new_var_name
    df_pat[new_var_name] = (df_pat[date_var] - df_pat['BirthDate']).dt.days

    min_day: int = df_pat[new_var_name].min()
    if min_day < 0:
        df_dict_pat.loc[df_dict_pat['Variable_Name'] == new_var_name, 'Remark'] = 'Unreasonably negative values detected'
        print(date_var, 'Negative')
    df_dict_pat.loc[df_dict_pat['Variable_Name'] == new_var_name, 'Variable_Type'] = 'Continuous'
df_pat.drop(columns=date_vars, inplace=True)

In [ ]:
########################################################################################################################
# [Patient-level, Part 4] Identify the problematic columns
########################################################################################################################
problem_cols: list[str] = ['AlcDrinksPerDay', 'CigPackYears'] + [col for col in df_pat.columns if 'DateKey' in col]

########################################################################################################################
# [Patient-level, Part 5] Perform the conversion from negative values to NaN
########################################################################################################################
for col in problem_cols:
    m: int = df_pat[df_pat[col] < 0].shape[0]
    if m == 0:
        continue

    n: int = df_pat[col].notna().sum()
    pct: float = (m * 100 / n)
    print(f'{col} {m} negative values ({pct}%)')

    df_pat.loc[df_pat[col] < 0, col] = np.nan
    assert df_pat[df_pat[col] < 0].shape[0] == 0

    df_dict_pat.loc[df_dict_pat['Variable_Name'] == col, 'Remark'] = \
        df_dict_pat.loc[df_dict_pat['Variable_Name'] == col, 'Remark'] + \
        f'; negative values (n={m}) converted to NaN.'


In [ ]:
########################################################################################################################
# [Patient-level, Part 6] Save the patient-level data
########################################################################################################################
df_pat.to_parquet(pat_path.replace('_v1', '_v2'))
print(f'Dimension of the updated patient-level dataset: {(df_pat.shape)}')
del df_pat
gc.collect()

In [ ]:
########################################################################################################################
# [Encounter-level, Part 1] Load the data
########################################################################################################################
enc_path: str = os.path.join(DATA_IN_DIR_PATH, 'Encounter_full_v1.parquet')
df_enc: pd.DataFrame = pd.read_parquet(enc_path)
print(f'Dimension of the encounter-level dataset: {df_enc.shape}')

In [ ]:
########################################################################################################################
# [Encounter-level, Part 2] Remove certain unneeded variables (due to small sample size)
########################################################################################################################
remove_keywords: list[str] = ['Score5Score', 'Score5DateKey', 'Score5Level',
                              'Score1Score', 'Score5DateKey', 'Score5Level',
                              'PainIntensityScore', 'PainIntensityDateKey',
                              'VASScore', 'VASDateKey',
                              'PainInterfScore', 'PainInterfDateKey',
                              'Pulmonary', 'Cortisol', 'FastingGlucose',
                              'EncDateKey']
enc_vars_to_remove: list[str] = [var for var in df_enc.columns if any(var.startswith(kw) for kw in remove_keywords)]
df_enc.drop(columns=enc_vars_to_remove, inplace=True, errors='ignore')
df_dict_enc = df_dict_enc[~df_dict_enc['Variable_Name'].isin(enc_vars_to_remove)]

In [ ]:
########################################################################################################################
# [Encounter-level, Part 3] For each of the remaining 5 Date variables, make it the days of difference from the EncDate
########################################################################################################################
date_vars: list[str] = df_dict_enc.loc[df_dict_enc['Variable_Type'] == 'Date', 'Variable_Name'].to_list()
date_vars.remove('EncDate')
for date_var in date_vars:
    new_var_name: str = f'{date_var}-EncDate'
    df_dict_enc.loc[df_dict_enc['Variable_Name'] == date_var, 'Variable_Name'] = new_var_name
    df_enc[new_var_name] = (df_enc[date_var] - df_enc['EncDate']).dt.days
    max_day: int = df_enc[new_var_name].max()
    if max_day > 0:
        df_dict_enc.loc[df_dict_enc['Variable_Name'] == new_var_name, 'Remark'] = 'Unreasonably positive values detected'
    df_dict_enc.loc[df_dict_enc['Variable_Name'] == new_var_name, 'Variable_Type'] = 'Continuous'
df_enc.drop(columns=date_vars, inplace=True)

In [ ]:
########################################################################################################################
# [Encounter-level, Part 4] For each of the remaining 4 Year variables, make it the years of difference from the EncDate
########################################################################################################################
year_vars: list[str] = df_dict_enc.loc[df_dict_enc['Variable_Type'] == 'Year', 'Variable_Name'].to_list()
for year_var in year_vars:
    new_var_name: str = f'{year_var}-EncDate'
    df_dict_enc.loc[df_dict_enc['Variable_Name'] == year_var, 'Variable_Name'] = new_var_name
    df_enc[new_var_name] = (
        df_enc[year_var].dt.year.astype('Int32')
        - df_enc['EncDate'].dt.year.astype('Int32')
    )
    max_year: int = df_enc[new_var_name].max()
    if max_year > 0:
        df_dict_enc.loc[df_dict_enc['Variable_Name'] == new_var_name, 'Remark'] = 'Unreasonably positive values detected'
    df_dict_enc.loc[df_dict_enc['Variable_Name'] == new_var_name, 'Variable_Type'] = 'Continuous'
df_enc.drop(columns=year_vars, inplace=True)

In [ ]:
########################################################################################################################
# [Encounter-level, Part 5] Save the encounter-level data
########################################################################################################################
df_enc.to_parquet(enc_path.replace('_v1', '_v2'))
print(f'Dimension of the updated encounter-level dataset: {(df_enc.shape)}')
del df_enc
gc.collect()

In [ ]:
########################################################################################################################
# Export the updated data dictionary
########################################################################################################################
with pd.ExcelWriter(DICT_OUT_FILE_PATH) as writer:
    df_dict_pat.to_excel(writer, sheet_name='Patient', index=False)
    df_dict_enc.to_excel(writer, sheet_name='Encounter', index=False)